In [19]:
!pip3 install anthropic python-dotenv --quiet

You should consider upgrading via the '/Users/elie/.pyenv/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.


In [20]:
import anthropic
import dotenv

dotenv.load_dotenv()

client = anthropic.Client()

MODEL_NAME = "claude-3-opus-20240229"

In [21]:
def multiply(num1, num2):
    return num1 * num2


In [22]:
result = client.messages.create(
    model=MODEL_NAME,
    messages=[{'role': 'user', 'content': 'multiply 321321 and 438219483 only respond with the resulting number'}],
    max_tokens=500,
)

# 14,081,438,016,68

In [23]:
result.content[0].text == "140809122497043"

False

In [111]:
multiply_tool = {
    "name": "multiply",
    "description": "Multiply two numbers",
    "input_schema": {
        "type": "object",
        "properties": {
            "num1": {
                "type": "integer",
                "description": "The first number"
            },
            "num2": {
                "type": "integer",
                "description": "The second number"
            }
        },
        "required": ["num1", "num2"]
    },
}

messages = [{'role': 'user',
             'content': 'multiply 321321 and 438219483 only respond with the resulting number'}]

In [112]:
result = client.messages.create(
    model=MODEL_NAME,
    messages=messages,
    max_tokens=1024,
    tools=[multiply_tool]
)

In [113]:
result.content

[TextBlock(text='<thinking>\nThe user has directly provided both of the required parameters (num1 and num2) for the "multiply" tool. No other tools are needed, and the parameter values are specified in the request, so we can proceed with calling the multiply tool without asking for any additional information.\n</thinking>', type='text'),
 ToolUseBlock(id='toolu_018jm9SouM9MFT8CRnaNWVMh', input={'num1': 321321, 'num2': 438219483}, name='multiply', type='tool_use')]

In [114]:
# result.content[0].input
# result.content[0].id

num1 = result.content[-1].input['num1']
num2 = result.content[-1].input['num2']

product = multiply(num1, num2)

messages.append({"role": "assistant", "content": result.content})

messages.append({
    'role': 'user',
    'content': [
        {
            "type": "tool_result",
            "tool_use_id": result.content[-1].id,
            "content": f" The product is {product}, which is the result of multiplying {num1} and {num2}. "
        }
    ]
})

In [115]:
messages

[{'role': 'user',
  'content': 'multiply 321321 and 438219483 only respond with the resulting number'},
 {'role': 'assistant',
  'content': [TextBlock(text='<thinking>\nThe user has directly provided both of the required parameters (num1 and num2) for the "multiply" tool. No other tools are needed, and the parameter values are specified in the request, so we can proceed with calling the multiply tool without asking for any additional information.\n</thinking>', type='text'),
   ToolUseBlock(id='toolu_018jm9SouM9MFT8CRnaNWVMh', input={'num1': 321321, 'num2': 438219483}, name='multiply', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_018jm9SouM9MFT8CRnaNWVMh',
    'content': ' The product is 140809122497043, which is the result of multiplying 321321 and 438219483. '}]}]

In [116]:
next_result = client.messages.create(
    model=MODEL_NAME,
    messages=messages,
    max_tokens=1024,
    tools=[multiply_tool]
)

In [117]:
next_result.content[0].text == "140809122497043"

True